In [6]:

import jax.numpy as jnp

import sys
sys.path.insert(1,'../scripts/')

import utils
import loader
import tqdm
%load_ext autoreload
%autoreload 2
import imp
imp.reload(loader)
import setup
import scipy.stats as sts
from matplotlib import pyplot as plt

import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:

params = {
    'file': '../data/',
    'tag': '2022_Q2_IBL_et_al_RepeatedSite',
    'probe': 'probe00',
    # 'sessions': [5,6,7,8,9,10,11,12],
    'areas': ['CA1','DG','LP','PO','VISa'],
    'props':{'train':.5,'test':.5,'validation':0},
    'seeds':{'train':0,'test':1,'validation':2},
    'n_neurons': None, # all neurons
    'n_trials': None, # all trials
    'pre_time':1,
    'post_time':1,
    'n_bins': 150,
    'align_to': 'stim',
    'train_trial_prop':.9, 
    'train_condition_prop':1, 
    'seed':0,
    'verbose': False,
    'bins_as_conds': False
}

dataloader = loader.IBLDataLoader(
    params,eids=setup.good_eids,parallel=False
)



local md5 mismatch on dataset: cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.brainLocationIds_ccf_2017.npy
local md5 mismatch on dataset: cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.mlapdv.npy
../data/2022_Q2_IBL_et_al_RepeatedSite/cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.brainLocationIds_ccf_2017.npy: 100%|██████████| 3.20k/3.20k [00:00<00:00, 9.38kB/s]
../data/2022_Q2_IBL_et_al_RepeatedSite/cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.mlapdv.npy: 100%|██████████| 4.74k/4.74k [00:00<00:00, 17.0kB/s]
local md5 mismatch on dataset: cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.brainLocationIds_ccf_2017.npy
local md5 mismatch on dataset: cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.mlapdv.npy
../data/2022_Q2_IBL_et_al_RepeatedSite/cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.brainLocationIds_ccf_201

In [8]:
xs,ys,rs,cs = dataloader.load_train_data()
ys = np.array(ys)

/var/folders/6w/snfd5vc50jb775fq8364m1d80000gn/T/ipykernel_32498/4188501355.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ys = np.array(ys)


In [9]:
# save data 
# with(open("25_avg_folds_stim_aligned.pkl","wb")) as f:
#     import pickle
#     pickle.dump([params,_ys,train_data_folds,test_data_folds],f)
    

In [10]:
params = {
    'file': '../data/',
    'tag': '2022_Q2_IBL_et_al_RepeatedSite',
    'probe': 'probe00',
    'sessions': [0,5,6],
    'areas': ['CA1','DG','LP','PO','VISa'],
    'props':{'train':.5,'test':.5,'validation':0},
    'seeds':{'train':0,'test':1,'validation':2},
    'n_neurons': None, # all neurons
    'n_trials': None, # all trials
    'pre_time':0,
    'post_time':0.4,
    'n_bins': 10,
    'align_to': 'response',
    'train_trial_prop':.9, 
    'train_condition_prop':1, 
    'seed':0,
    'verbose': False,
    'bins_as_conds': True
    }

dataloader = loader.IBLDataLoader(
         params,eids=setup.good_eids
)


local md5 mismatch on dataset: cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.brainLocationIds_ccf_2017.npy
local md5 mismatch on dataset: cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.mlapdv.npy
../data/2022_Q2_IBL_et_al_RepeatedSite/cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.brainLocationIds_ccf_2017.npy: 100%|██████████| 3.20k/3.20k [00:00<00:00, 11.4kB/s]
../data/2022_Q2_IBL_et_al_RepeatedSite/cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.mlapdv.npy: 100%|██████████| 4.74k/4.74k [00:00<00:00, 18.3kB/s]
local md5 mismatch on dataset: cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.brainLocationIds_ccf_2017.npy
local md5 mismatch on dataset: cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.mlapdv.npy
../data/2022_Q2_IBL_et_al_RepeatedSite/cortexlab/Subjects/KS074/2021-11-22/001/alf/probe00/pykilosort/channels.brainLocationIds_ccf_201

In [11]:
n_folds = 1

train_data_folds, test_data_folds = dataloader.new_folds(n_folds)

__train_data_folds = []
__test_data_folds = []

for i in range(n_folds):
    _,ys,_,cs = train_data_folds[i]
    _,ys_t,_,cs_t = test_data_folds[i]

    # filter bad neurons/sessions
    min_neurons = 100
    min_trials = 20

    valid = [i for i in range(len(ys)) if ys[i].shape[2] >= min_neurons and ys[i].shape[0] >= min_trials]
    ys = [y[:,:,jnp.argsort(y.mean(0).std(0))[:100]] for y in ys]

    ys = [ys[i].mean(0) for i in valid]
    cs = [cs[i].mean(0) for i in valid]

    ys_t = [ys_t[i].mean(0) for i in valid]
    cs_t = [cs_t[i].mean(0) for i in valid]

    S = len(ys)
    __train_data_folds.append([ys,cs])
    __test_data_folds.append([ys_t,cs_t])

# # save pickle
# import pickle
# with open("100_folds_1_time_point.pkl","wb") as f:
#     pickle.dump([params,__train_data_folds,__test_data_folds],f)

In [14]:
valid

[1, 2, 3, 4, 8, 10, 13, 14, 15, 16, 19, 21, 22, 23, 24, 25, 26, 29, 31, 32]

In [12]:
# xs,ys,rs,cs = dataloader.load_train_data()

# valid_cs = np.array([cs[i] for i in valid])
# import pickle
# with open("raw_behavior.pkl","wb") as f:
#     pickle.dump(cs,f)